In [1]:
import sys
from pathlib import Path

# Use python-dotenv to load environment variables from .env file
from dotenv import load_dotenv

load_dotenv()


from langchain.globals import set_verbose, set_debug

# set_verbose(True)
# set_debug(True)

# Add the src directory to Python path
src_path = str(Path.cwd().parent / "src")
if src_path not in sys.path:
    sys.path.append(src_path)


In [ ]:
from utils.config import get_settings

settings = get_settings()
settings


In [ ]:
# Invoke the full graph
# Create agents
import uuid

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

from services.agents.analyst import ANALYST_SYSTEM_MESSAGE, DataAnalystAgent
from services.agents.developer import DEVELOPER_SYSTEM_MESSAGE, DeveloperAgent
from services.agents.graph import create_agent_graph
from services.agents.planner import PLANNER_SYSTEM_MESSAGE, PlannerAgent
from services.agents.supervisor import SUPERVISOR_SYSTEM_MESSAGE, SupervisorAgent
from services.agents.tools import (
    GetRepoCommitsTool,
    GetRepoIssuesTools,
    GetRepoPullRequestsTool,
)
from services.gql.client import Client

client = Client(
    url=str(settings.github_graphql_url),
    headers={"Authorization": settings.codegen_gh_auth.get_secret_value()},
)
llm = AzureChatOpenAI(
    api_version=settings.azure_openai_api_version,
    azure_endpoint=settings.azure_openai_endpoint,
    model="gpt-4o",
    api_key=settings.azure_openai_api_key.get_secret_value(),
)

llm_mini = AzureChatOpenAI(
    api_version=settings.azure_openai_api_version,
    azure_endpoint=settings.azure_openai_endpoint,
    model="gpt-4o-mini",
    api_key=settings.azure_openai_api_key.get_secret_value(),
)

supervisor = SupervisorAgent(
    llm=llm_mini,
    tools=[],
    system_message=SUPERVISOR_SYSTEM_MESSAGE,
    team_members=["planner", "analyst", "developer"],
)

planner = PlannerAgent(
    llm=llm,
    tools=[],
    system_message=PLANNER_SYSTEM_MESSAGE,
)

analyst_tools = [
    GetRepoIssuesTools(client=client),
    GetRepoCommitsTool(client=client),
    GetRepoPullRequestsTool(client=client),
]
analyst = DataAnalystAgent(
    llm=llm,
    tools=analyst_tools,
    system_message=ANALYST_SYSTEM_MESSAGE,
)

developer = DeveloperAgent(
    llm=llm,
    tools=[],
    system_message=DEVELOPER_SYSTEM_MESSAGE,
)

# Create the graph
graph = create_agent_graph(supervisor, planner, analyst, developer)

# Initialize the context
context = {
    "message": "Create a line chart showing the number of commits over time",
    "owner": "langchain-ai",
    "repo": "langchain",
    "chat_history": [
        HumanMessage(
            id=str(uuid.uuid4()),
            content="Create a line chart showing the number of commits over time",
            name="user",
        )
    ],
}

# Execute the graph
final_context = graph.invoke(context)

print(final_context["developer_output"].typescript_code)

In [3]:
data_sample = [
    {
        "committed_date": "2024-11-06T20:04:09Z",
        "authored_date": "2024-11-06T20:04:09Z",
        "author": "Eric Pinzur",
        "message": "community: added Document.id support to opensearch vectorstore (#27945)\n\nDescription:\r\n* Added support of Document.id on OpenSearch vector store\r\n* Added tests cases to match",
        "changed_files_if_available": 2,
        "additions": 55,
        "deletions": 21,
        "status": "SUCCESS",
    },
    {
        "committed_date": "2024-11-06T19:55:00Z",
        "authored_date": "2024-11-06T19:55:00Z",
        "author": "Hammad Randhawa",
        "message": 'docs: Completed sentence under the heading  "Instantiating a Browser … (#27944)\n\n…Toolkit" in "playwright.ipynb" integration.\r\n\r\n- Completed the incomplete sentence in the Langchain Playwright\r\ndocumentation.\r\n\r\n- Enhanced documentation clarity to guide users on best practices for\r\ninstantiating browser instances with Langchain Playwright.\r\n\r\nExample before:\r\n> "It\'s always recommended to instantiate using the from_browser method\r\nso that the\r\n\r\nExample after:\r\n> "It\'s always recommended to instantiate using the `from_browser`\r\nmethod so that the browser context is properly initialized and managed,\r\nensuring seamless interaction and resource optimization."\r\n\r\nCo-authored-by: Erick Friis <erick@langchain.dev>',
        "changed_files_if_available": 1,
        "additions": 1,
        "deletions": 1,
        "status": "SUCCESS",
    },
    {
        "committed_date": "2024-11-06T18:06:47Z",
        "authored_date": "2024-11-06T18:06:47Z",
        "author": "Bagatur",
        "message": "core[patch]: make oai tool description optional (#27756)",
        "changed_files_if_available": 2,
        "additions": 174,
        "deletions": 23,
        "status": "SUCCESS",
    },
    {
        "committed_date": "2024-11-06T17:50:07Z",
        "authored_date": "2024-11-06T17:50:07Z",
        "author": "Bagatur",
        "message": "docs: document init_chat_model standard params (#27812)",
        "changed_files_if_available": 1,
        "additions": 10,
        "deletions": 1,
        "status": "SUCCESS",
    },
    {
        "committed_date": "2024-11-06T14:58:16Z",
        "authored_date": "2024-11-06T14:58:16Z",
        "author": "Dobiichi-Origami",
        "message": "community: re-arrange function call message parse logic for Qianfan (#27935)\n\nthe [PR](https://github.com/langchain-ai/langchain/pull/26208) two month\r\nago has a potential bug which causes malfunction of `tool_call` for\r\n`QianfanChatEndpoint` waiting for fix",
        "changed_files_if_available": 1,
        "additions": 3,
        "deletions": 7,
        "status": "SUCCESS",
    },
    {
        "committed_date": "2024-11-06T00:43:11Z",
        "authored_date": "2024-11-06T00:43:11Z",
        "author": "Erick Friis",
        "message": "infra: starter codeowners file (#27929)",
        "changed_files_if_available": 1,
        "additions": 2,
        "deletions": 0,
        "status": "SUCCESS",
    },
    {
        "committed_date": "2024-11-05T23:02:24Z",
        "authored_date": "2024-11-05T23:02:24Z",
        "author": "ccurme",
        "message": "openai[patch]: release 0.2.6 (#27924)\n\nSome additions in support of [predicted\r\noutputs](https://platform.openai.com/docs/guides/latency-optimization#use-predicted-outputs)\r\nfeature:\r\n- Bump openai sdk version\r\n- Add integration test\r\n- Add example to integration docs\r\n\r\nThe `prediction` kwarg is already plumbed through model invocation.",
        "changed_files_if_available": 4,
        "additions": 150,
        "deletions": 12,
        "status": "SUCCESS",
    },
    {
        "committed_date": "2024-11-05T20:55:38Z",
        "authored_date": "2024-11-05T20:55:38Z",
        "author": "Erick Friis",
        "message": "standard-tests: ci pipeline (#27923)",
        "changed_files_if_available": 5,
        "additions": 71,
        "deletions": 7,
        "status": "SUCCESS",
    },
]


In [ ]:
from langchain_openai import AzureChatOpenAI

from services.agents.developer import DEVELOPER_SYSTEM_MESSAGE, DeveloperAgent
from services.agents.types import AnalystOutput, PlannerOutput, TechnicalSpecs
from services.gql.client import Client

client = Client(
    url=str(settings.github_graphql_url),
    headers={"Authorization": settings.codegen_gh_auth.get_secret_value()},
)

# Create the array of tools, that contains the tools for the analyst agent
# He should be able to query GitHub issues, Pull requests, and commits

# Initialize the LLM
llm = AzureChatOpenAI(
    api_version=settings.azure_openai_api_version,
    azure_endpoint=settings.azure_openai_endpoint,
    model="gpt-4o",
    api_key=settings.azure_openai_api_key.get_secret_value(),
)

# Initialize the planner agent
developer = DeveloperAgent(
    llm=llm,
    system_message=DEVELOPER_SYSTEM_MESSAGE,
    tools=[],
)

# Test context
context = {
    "message": "Create a line chart showing the number of commits over time",
    "owner": "langchain-ai",
    "repo": "langchain",
    "planner_output": PlannerOutput(
        requirements=[
            "The system shall generate a line chart displaying the number of commits over time.",
            "The data shall be fetched from the GitHub repository langchain-ai/langchain.",
            "The user shall be able to specify the time range for which the data is displayed.",
            "The chart shall update dynamically as new commits are made.",
        ],
        acceptance_criteria=[],
        technical_specs=TechnicalSpecs(
            chart_type="line", data_format="", technical_constraints=[]
        ),
        error_message=None,
    ),
    "analyst_output": AnalystOutput(
        data_sample=data_sample,
        file_path="../data/langchain-ai/langchain/data.json",
        error_message=None,
    ),
}

# Execute the agent
result = await developer.execute(context)
print(result)


In [ ]:
from langchain_openai import AzureChatOpenAI

from services.agents.analyst import ANALYST_SYSTEM_MESSAGE, DataAnalystAgent
from services.agents.tools import (
    GetRepoCommitsTool,
    GetRepoIssuesTools,
    GetRepoPullRequestsTool,
)
from services.agents.types import PlannerOutput, TechnicalSpecs
from services.gql.client import Client

client = Client(
    url=str(settings.github_graphql_url),
    headers={"Authorization": settings.codegen_gh_auth.get_secret_value()},
)

# Create the array of tools, that contains the tools for the analyst agent
# He should be able to query GitHub issues, Pull requests, and commits
tools = [
    GetRepoIssuesTools(client=client),
    GetRepoCommitsTool(client=client),
    GetRepoPullRequestsTool(client=client),
]

# Initialize the LLM
llm = AzureChatOpenAI(
    api_version=settings.azure_openai_api_version,
    azure_endpoint=settings.azure_openai_endpoint,
    model="gpt-4o",
    api_key=settings.azure_openai_api_key.get_secret_value(),
)

# Initialize the planner agent
analyst = DataAnalystAgent(
    llm=llm,
    system_message=ANALYST_SYSTEM_MESSAGE,
    tools=tools,
)

# Test context
context = {
    "message": "Create a line chart showing the number of commits over time",
    "owner": "langchain-ai",
    "repo": "langchain",
    "planner_output": PlannerOutput(
        requirements=[
            "The system shall generate a line chart displaying the number of commits over time.",
            "The data shall be fetched from the GitHub repository langchain-ai/langchain.",
            "The user shall be able to specify the time range for which the data is displayed.",
            "The chart shall update dynamically as new commits are made.",
        ],
        acceptance_criteria=[],
        technical_specs=TechnicalSpecs(
            chart_type="line", data_format="", technical_constraints=[]
        ),
        error_message=None,
    ),
}

# Execute the agent
result = await analyst.execute(context)
print(result)


In [ ]:
from langchain_openai import AzureChatOpenAI

from services.agents.planner import PLANNER_SYSTEM_MESSAGE, PlannerAgent
from services.agents.types import PlannerOutput, TechnicalSpecs

# Initialize the LLM
llm = AzureChatOpenAI(
    api_version=settings.azure_openai_api_version,
    azure_endpoint=settings.azure_openai_endpoint,
    model="gpt-4o",
    api_key=settings.azure_openai_api_key.get_secret_value(),
)

# Initialize the planner agent
planner = PlannerAgent(llm=llm, system_message=PLANNER_SYSTEM_MESSAGE, tools=[])

# Test context
context = {
    "message": "Create a line chart showing the number of commits over time",
    "owner": "langchain-ai",
    "repo": "langchain",
}

# Execute the agent
result = planner.execute(context)
print(result)